In [ ]:
# # imports
# import psycopg2
# import numpy as np
# import pandas as pd
# import json
# from sqlalchemy import create_engine
# import matplotlib.pyplot as plt
# from scipy import interpolate

# resolution = 100 # can head to config file

# # access creds
# with open('psql_key.json') as file:
#     creds = json.load(file)

# # Create an SQLAlchemy engine with the PostgreSQL database URI
# engine = create_engine(f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['database']}")

# # these diag codes were taken from table mimiciv_hosp.d_icd_diagnoses
# hf_codes = "('39891','40200','40201','40210','40211','40290','40291','40400','40401','40402','40403','40410','40411','40412','40413','40490','40491','40492','40493','4280','4281','42820','42821','42822','42823','42830','42831','42832','42833','42840','42841','42842','42843','4289','E8726','E8745','I0981','I110','I119','I130','I131','I1310','I1311','I132','I50','I501','I502','I5020','I5021','I5022','I5023','I503','I5030','I5031','I5032','I5033','I504','I5040','I5041','I5042','I5043','I508','I5081','I50810','I50811','I50812','I50813','I50814','I5082','I5083','I5084','I5089','I509','I9713','I97130','I97131','T8622','T8632','Y625')"

# # get all admissions related to HF codes
# query = f'''
# select a.* from (select * from mimiciv_hosp.diagnoses_icd where icd_code in {hf_codes}) d
# inner join (select * from mimiciv_hosp.admissions) a
# on a.hadm_id = d.hadm_id
# '''
# all_hf_pats = pd.read_sql_query(query , engine)

# # QC
# print(all_hf_pats['subject_id'].nunique())
# # all_hf_pats.head(2)

# # Count of times patients have been admitted with a HF diagnosis

# hist_df = all_hf_pats.groupby('subject_id').agg({'admittime':'nunique'}).reset_index().groupby('admittime').agg({'subject_id':'nunique'}).reset_index()
# hist_df.columns = ['num_admits' , 'patients']
# hist_df['cumulative_patients'] = hist_df['patients'].cumsum()
# # hist_df.head()

# # Retain all patients
# # Add num of ICU admits / num of admissions as covariates
# # Time of survival -> from last ICU admission to deathtime / OR first ICU admission?
# # Uncensored patients (those that experienced the event, i.e. died)
# # Find latest ICU admission
# tos_hf = all_hf_pats.groupby('subject_id').agg({'admittime':'min' , 'deathtime':'max' , 'dischtime':'max'})
# event_pats = tos_hf[tos_hf['deathtime'].notnull()].copy()
# event_pats['time_to_event'] = event_pats['deathtime'] - event_pats['admittime']
# event_pats['time_to_event'] = event_pats['time_to_event'] / np.timedelta64(1, 'D') # convert to hours
# event_pats['Uncensored'] = 1
# # event_pats.head(3)

# # who are the censored patients?
# non_event_pats = tos_hf[tos_hf['deathtime'].isnull()].copy()# For censored patients, their last follow-up was essentially their discharge time
# non_event_pats['time_to_event'] = non_event_pats['dischtime'] - non_event_pats['admittime']
# non_event_pats['time_to_event'] = non_event_pats['time_to_event'] / np.timedelta64(1, 'D') # convert to hours
# # non_event_pats['time_to_event'] = non_event_pats['time_to_event'] + 365 # is this necessary?
# non_event_pats['Uncensored'] = 0
# # non_event_pats.head(3)

# pats = pd.concat([non_event_pats[['time_to_event' , 'Uncensored']] , event_pats[['time_to_event' , 'Uncensored']]] , axis = 0)
# pats.reset_index(inplace = True)
# pats.sort_values('subject_id' , inplace = True)
# # pats.head()

# # Try to get vital signs

# query = f'''
# select * from mimiciv_ed.vitalsign
# '''
# vitalsign = pd.read_sql_query(query , engine)
# # pats_vitalsign = vitalsign.merge(pats['subject_id'] , on = 'subject_id' , how = 'inner')
# # pats_vitalsign_list = pats_vitalsign.subject_id.unique()
# # print(f'unique patients: {pats_vitalsign.subject_id.nunique()}')

# # # Get survival
# # pats_survival = pats.merge(vitalsign['subject_id'] , on = 'subject_id' , how = 'inner')
# # print(f'unique patients: {pats_survival.subject_id.nunique()}')
# # pats_survival.tail()

# pats_all_metrics = vitalsign.merge(pats , on = 'subject_id' , how = 'inner')
# print(f'unique patients: {pats_all_metrics.subject_id.nunique()}')
# # pats_all_metrics.tail()